<a href="https://colab.research.google.com/github/amitagar1/EVA2.0/blob/master/EVA2_Session13_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)
from keras.optimizers import SGD 

Using TensorFlow backend.


In [2]:
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
#n_train, n_test = X_train.shape[0], X_test.shape[0]
#img_size = X_train.shape[1]
#n_classes = y_train.max() + 1

# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
num_classes = 10
# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 11s 0us/step


#Data Normalization

In [0]:
x_train_mean = np.mean(x_train, axis=(0,1,2))
x_train_std = np.std(x_train, axis=(0,1,2))
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

In [0]:
x_train.shape

(50000, 32, 32, 3)

In [0]:
def pad(x, pad=4):
    return np.pad(x, ((0,0), (pad,pad),(pad,pad),(0,0)), mode='reflect')

In [0]:
X_train = pad(X_train_0)
X_test= pad(X_test_0)

In [0]:
X_train.shape

(50000, 32, 32, 3)

# Data Agumentation - Work In Progress

In [0]:
# Data Preparation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    horizontal_flip=(True, .5),
    #preprocessing_function=get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3,
                 # v_l=0, v_h=255, pixel_level=False)
    )


In [0]:
def crop_generator(xy_gen, crop_length):
    while True:
        x, y = next(xy_gen)
        batch_crops = np.zeros((5, crop_length, crop_length, 3))
        batch_ys = np.tile(y, (5, 1))  # repeat 'y' 5 times
        for i in range(5):
            batch_crops[i] = random_crop(x, (crop_length, crop_length))
        yield (batch_crops, batch_ys)
        
def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

train_xy_generator = train_datagen.flow_from_directory(
    DATASET_PATH + '/train',
    target_size=None,               # don't rescale original image
    class_mode='categorical',
    shuffle=True,
    batch_size=1)

In [0]:
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

# ResNet Implementation

In [0]:
def identity_block(X, f, filters, stage, block, s):
# defining name basis
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
# Retrieve Filters
  F1, F2 = filters   
# Save the input value. You'll need this later to add back to the main path. 
  X_shortcut = X    
# First Block
  X = Conv2D(filters = F1, kernel_size = (f, f), strides = (s,s), padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + 'a')(X)
  X = Activation('relu')(X)    
# Second Block component of main path (≈2 lines)
  X = Conv2D(filters = F2, kernel_size = (1, 1), strides = (1,1), padding = 'same', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + 'b')(X)
# Addition Block
  X_shortcut = Conv2D(filters = F2, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)       
  return X

In [0]:
def ResNet_model(input_shape=(32, 32, 3), classes=10):
 # Define the input as a tensor with shape input_shape
  X_input = Input(input_shape)
# Zero-Padding
 # X = ZeroPadding2D((3, 3))(X_input)
# Conv_Layer
  X = Conv2D(64, (3, 3), strides=(1, 1), padding = 'same', name='conv1', kernel_initializer=glorot_uniform(seed=0))(X_input)
  X = BatchNormalization(axis=3, name='bn_conv1')(X)
  X = Activation('relu')(X)
#ResLayer_1_2Blocks_addition
  X = identity_block(X, 3, [64,128],s=1, stage=2, block='a')  
#ResLayer_2_2Blocks_addition
  X = identity_block(X, 3, [128,256],s=2, stage=3, block='a')  
# ResLayer_3_2Blocks_addition
  X = identity_block(X, 3, [256,512 ],s=2, stage=4, block='a')
# ResLayer_4_2Blocks_addition
  X = identity_block(X, 3, [512,512],s=1, stage=5, block='a')
  X = AveragePooling2D((8,8), name="avg_pool")(X)
# output layer
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X) 
# Create model
  model = Model(inputs = X_input, outputs = X, name='ResNet18')
  return model

In [0]:
model = ResNet50(input_shape = (32, 32, 3), classes = 10)

In [0]:
model.summary()

Model: "ResNet18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [0]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-1, decay = 5e-4, momentum=0.9),
              metrics=['accuracy'])


In [0]:
import datetime
batch_size= 128
epochs = 10
start = datetime.datetime.now()
model_details = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), 
                    steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs,
                    shuffle=True,
                    #callbacks= [checkpoint_1],#LearningRateScheduler(step_decay)],                                    
                    validation_data=(x_test, y_test),
                    verbose=1)
end = datetime.datetime.now()
print ("Model took %0.2f seconds to train"%(end - start))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
391/390 [==============================] - 92s 235ms/step - loss: 1.5708 - acc: 0.4276 - val_loss: 1.2687 - val_acc: 0.5325
Epoch 2/10
391/390 [==============================] - 84s 216ms/step - loss: 1.1356 - acc: 0.5895 - val_loss: 0.9542 - val_acc: 0.6599
Epoch 3/10
391/390 [==============================] - 85s 216ms/step - loss: 0.9270 - acc: 0.6716 - val_loss: 0.8224 - val_acc: 0.7081
Epoch 4/10
391/390 [==============================] - 84s 216ms/step - loss: 0.8046 - acc: 0.7153 - val_loss: 0.7559 - val_acc: 0.7328
Epoch 5/10
391/390 [==============================] - 84s 216ms/step - loss: 0.7058 - acc: 0.7532 - val_loss: 0.6213 - val_acc: 0.7868
Epoch 6/10
391/390 [==============================] - 84s 216ms/step - loss: 0.6312 - acc: 0.7802 - val_loss: 0.5913 - val_acc: 0.7936
Epoch 7/10
391/390 [==============================] - 84s 216ms/step - loss: 0.5807 - acc: 0.7

TypeError: ignored